In [2]:
from utils import MidiProcessor, prepare_data, array_to_midi, concat_all_midi_to_df, get_good_files
from model import build_bidirectional_lstm
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.utils import np_utils
from keras.models import load_model

In [5]:
input_window_len = 32
pred_steps = 1

In [6]:
files = get_good_files()

df, tracks_len_list = concat_all_midi_to_df(files, return_tracks_len_list=True)

42 drum loops
31 percussion instruments
172314 32-notes


In [7]:
X_train, y_train, X_test, y_test, instruments_list = prepare_data(df, input_window_len=input_window_len, pred_steps=1, overlaps=0, train_test_split=0.1,
                                                                  tracks_len_list=tracks_len_list, max_instruments=6)

In [6]:
y_test

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 0]])

In [8]:
n_feature = X_train.shape[2]
output_shape = y_train.shape[1]
model = build_bidirectional_lstm(input_window_len=input_window_len, n_feature=n_feature, output_shape=output_shape, dropout=0.5, neurons_per_layer=[32, 32, 32])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32, 32)            4992      
_________________________________________________________________
dropout (Dropout)            (None, 32, 32)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               106400    
_________________________________________________________________
batch_normalization (BatchNo (None, 200)               800       
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 1206      
_________________________________________________________________
activation (Activation)      (None, 6)                 0

In [24]:
es = EarlyStopping(monitor='val_loss', patience=50)
mc = ModelCheckpoint(filepath='./bidirectional_1st_try.h5',
                     monitor='val_loss', verbose=1, save_best_only=True)
callbacks = [es, mc]

history = model.fit(X_train, y_train, epochs=50, callbacks=callbacks,
                    validation_split=0.1, verbose=1, shuffle=False)